In [4]:
from dotenv import load_dotenv
import sys
sys.path.append('..')


load_dotenv()

True

In [5]:
from src.vector import EmbeddingManager, OpenAIEmbeddingProvider, FAISSVectorStore, EmbeddingConfig
from src.vector import create_file_content_map
import os

file_content_map = create_file_content_map(
    root_dir=os.path.join("..", "src", "watcher"), accept_patterns=["*.py"])


embedding_provider = OpenAIEmbeddingProvider(
    config=EmbeddingConfig(model_name="text-embedding-3-small"))
vector_store = FAISSVectorStore(dimension=1536, index_path="faiss_index.bin")

embedding_manager = EmbeddingManager(
    embedding_provider=embedding_provider,
    vector_store=vector_store,
)


texts = list(file_content_map.values())
metadatas = [{"source": file_path} for file_path in file_content_map.keys()]

await embedding_manager.add_texts(texts=texts, metadatas=metadatas)

Found 4 valid files


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 0.466326 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 0.935061 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
from src.query import QueryProcessor
from src.prompt.prompt import PromptType, PromptProviderType
from langchain_openai import ChatOpenAI
from src.cmd import create_output_manager


query_processor = QueryProcessor   (embedding_manager=embedding_manager,
                                              llm=ChatOpenAI(model="gpt-4o-mini"))


final_result = ""

output_manager = create_output_manager()

async for result in query_processor.process_query(query="how file watching is implemented", prompt_type=PromptType.AGGREGATE, prompt_provider=PromptProviderType.SEMANTIC):
    await output_manager.print_stream(result)




INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 401 Unauthorized"
ERROR:src.vector.embedding:Error generating query embedding: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************50cA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


Error performing similarity search: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************50cA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Error processing query: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************50cA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************50cA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [5]:
with open("result.md", "w") as f:
    f.write(final_result)

